<a href="https://colab.research.google.com/github/yusufshihata/InternIntelligence_NER/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate

## Load the Dataset

In [ ]:
dataset = load_dataset("conll2003")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
labels = dataset['train'].features['ner_tags'].feature.names

labels

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
training_data = []
def map_word_label(data):
    for i in range(len(data)):
        token2label = {"tokens": [], "labels": []}
        token2label['tokens'], token2label['labels'] = data[i]['tokens'], data[i]['ner_tags']
        training_data.append(token2label)

In [ ]:
map_word_label(dataset['train'])

In [ ]:
training_data[0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'labels': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [1]:
!pip install datasets seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=65ae6c39163bc1f3e802c99f6d21174e2cd660889cf0deece1375e5c3a1afe43
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

# Dataset Handler Class
class DatasetPreprocessor:
    def __init__(self, tokenizer, dataset_name="conll2003"):
        self.dataset = load_dataset(dataset_name)
        self.tokenizer = tokenizer
        self.label_list = self.dataset["train"].features["ner_tags"].feature.names
        self.num_labels = len(self.label_list)

    def tokenize_and_align_labels(self, examples):
        tokenized_inputs = self.tokenizer(
            examples["tokens"], truncation=True, is_split_into_words=True,
            padding="max_length", max_length=25
        )

        all_labels = []
        for i, labels in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(labels[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            all_labels.append(label_ids)
        tokenized_inputs["labels"] = all_labels
        return tokenized_inputs

    def get_tokenized_dataset(self):
        return self.dataset.map(self.tokenize_and_align_labels, batched=True)

# Strategy Pattern for Metrics Computation
class MetricsComputer:
    def __init__(self, label_list):
        self.metric = evaluate.load("seqeval")
        self.label_list = label_list

    def compute(self, p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        true_predictions = []
        true_labels = []
        for prediction, label in zip(predictions, labels):
            pred_tags = []
            true_tags = []
            for p_val, l_val in zip(prediction, label):
                if l_val != -100:
                    pred_tags.append(self.label_list[p_val])
                    true_tags.append(self.label_list[l_val])
            true_predictions.append(pred_tags)
            true_labels.append(true_tags)

        results = self.metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

# NER Trainer Class
class NERTrainer:
    def __init__(self, model, tokenizer, train_dataset, eval_dataset, metric_computer):
        self.model = model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.metric_computer = metric_computer
        self.data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        self.training_args = TrainingArguments(
            output_dir="./results", evaluation_strategy="epoch", learning_rate=2e-5,
            per_device_train_batch_size=16, per_device_eval_batch_size=16,
            num_train_epochs=3, weight_decay=0.01, logging_dir="./logs",
        )

    def train(self):
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
            compute_metrics=self.metric_computer.compute,
        )
        trainer.train()

    def ner_inference(sentence):
        # Naively split the sentence into words
        words = sentence.split()

        # First, tokenize without return_tensors to retain alignment info
        encoding = tokenizer(
            words,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=25,
            return_offsets_mapping=True
        )

        # Get word IDs from the encoding
        token_word_ids = encoding.word_ids(batch_index=0)

        # Convert input_ids to tensors and move to model's device
        inputs = {
            k: torch.tensor(v).unsqueeze(0).to(model.device)
            for k, v in encoding.items() if k != "offset_mapping"
        }

        # Run the model inference
        with torch.no_grad():
            outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)[0].tolist()

        # Align predictions: only take the first token prediction for each word
        predicted_labels = []
        current_word_idx = None
        for idx, word_idx in enumerate(token_word_ids):
            if word_idx is None:
                continue
            if word_idx != current_word_idx:
                predicted_label = dataset_handler.label_list[predictions[idx]]
                predicted_labels.append(predicted_label)
                current_word_idx = word_idx

        return list(zip(words, predicted_labels))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
dataset_preprocessor = DatasetPreprocessor(tokenizer)
tokenized_dataset = dataset_preprocessor.get_tokenized_dataset()

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=dataset_preprocessor.num_labels)
metric_computer = MetricsComputer(dataset_preprocessor.label_list)
trainer = NERTrainer(
    model, tokenizer, tokenized_dataset["train"], tokenized_dataset["validation"], metric_computer
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yusufshihata2006 (yusufshihata2006-kafr-el-sheikh-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.176500,0.048130,0.928557,0.932172,0.930361,0.987118
2,0.032100,0.046864,0.938086,0.943852,0.940960,0.989100
3,0.015300,0.047019,0.940384,0.943852,0.942115,0.989308


In [2]:
import numpy as np
import torch
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

In [3]:
class DatasetPreprocessor:
    def __init__(self, tokenizer, dataset_name="conll2003"):
        self.dataset = load_dataset(dataset_name)
        self.tokenizer = tokenizer
        self.label_list = self.dataset["train"].features["ner_tags"].feature.names
        self.num_labels = len(self.label_list)

    def tokenize_and_align_labels(self, examples):
        tokenized_inputs = self.tokenizer(
            examples["tokens"], truncation=True, is_split_into_words=True,
            padding="max_length", max_length=25
        )

        all_labels = []
        for i, labels in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(labels[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            all_labels.append(label_ids)
        tokenized_inputs["labels"] = all_labels
        return tokenized_inputs

    def get_tokenized_dataset(self):
        return self.dataset.map(self.tokenize_and_align_labels, batched=True)

In [4]:
class MetricsComputer:
    def __init__(self, label_list):
        self.metric = evaluate.load("seqeval")
        self.label_list = label_list

    def compute(self, p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        true_predictions = []
        true_labels = []
        for prediction, label in zip(predictions, labels):
            pred_tags = []
            true_tags = []
            for p_val, l_val in zip(prediction, label):
                if l_val != -100:
                    pred_tags.append(self.label_list[p_val])
                    true_tags.append(self.label_list[l_val])
            true_predictions.append(pred_tags)
            true_labels.append(true_tags)

        results = self.metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [9]:
class NERTrainer:
    def __init__(self, model, tokenizer, train_dataset, eval_dataset, metric_computer):
        self.model = model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.metric_computer = metric_computer
        self.data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        self.training_args = TrainingArguments(
            output_dir="./results", evaluation_strategy="epoch", learning_rate=2e-5,
            per_device_train_batch_size=16, per_device_eval_batch_size=16,
            num_train_epochs=3, weight_decay=0.01, logging_dir="./logs",
        )

    def train(self):
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
            compute_metrics=self.metric_computer.compute,
        )
        trainer.train()

    def inference(self, sentence):
        # Naively split the sentence into words
        words = sentence.split()

        # First, tokenize without return_tensors to retain alignment info
        encoding = tokenizer(
            words,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=25,
            return_offsets_mapping=True
        )

        # Get word IDs from the encoding
        token_word_ids = encoding.word_ids(batch_index=0)

        # Convert input_ids to tensors and move to model's device
        inputs = {
            k: torch.tensor(v).unsqueeze(0).to(model.device)
            for k, v in encoding.items() if k != "offset_mapping"
        }

        # Run the model inference
        with torch.no_grad():
            outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)[0].tolist()

        # Align predictions: only take the first token prediction for each word
        predicted_labels = []
        current_word_idx = None
        for idx, word_idx in enumerate(token_word_ids):
            if word_idx is None:
                continue
            if word_idx != current_word_idx:
                predicted_label = dataset_preprocessor.label_list[predictions[idx]]
                predicted_labels.append(predicted_label)
                current_word_idx = word_idx

        return list(zip(words, predicted_labels))

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
dataset_preprocessor = DatasetPreprocessor(tokenizer)
tokenized_dataset = dataset_preprocessor.get_tokenized_dataset()

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=dataset_preprocessor.num_labels)
metric_computer = MetricsComputer(dataset_preprocessor.label_list)
trainer = NERTrainer(
    model, tokenizer, tokenized_dataset["train"], tokenized_dataset["validation"], metric_computer
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer.train()

<ipython-input-9-aaf61f3861a1>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.178500,0.046993,0.924513,0.933607,0.929038,0.987326
2,0.032600,0.044488,0.934066,0.940574,0.937309,0.988891
3,0.015000,0.046103,0.935733,0.942828,0.939267,0.988891


In [14]:
example_sentence = input("Enter your sentence: ")
result = trainer.inference(example_sentence)
print("NER predictions:")
for word, label in result:
    print(f"{word}: {label}")

Enter your sentence: Intern Intelligence is an Egyptian Company
NER predictions:
Intern: B-ORG
Intelligence: I-ORG
is: O
an: O
Egyptian: B-MISC
Company: O


In [ ]:
def ner_inference(sentence):
    # Naively split the sentence into words
    words = sentence.split()

    # First, tokenize without return_tensors to retain alignment info
    encoding = tokenizer(
        words,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=25,
        return_offsets_mapping=True
    )

    # Get word IDs from the encoding
    token_word_ids = encoding.word_ids(batch_index=0)

    # Convert input_ids to tensors and move to model's device
    inputs = {
        k: torch.tensor(v).unsqueeze(0).to(model.device)
        for k, v in encoding.items() if k != "offset_mapping"
    }

    # Run the model inference
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)[0].tolist()

    # Align predictions: only take the first token prediction for each word
    predicted_labels = []
    current_word_idx = None
    for idx, word_idx in enumerate(token_word_ids):
        if word_idx is None:
            continue
        if word_idx != current_word_idx:
            predicted_label = dataset_handler.label_list[predictions[idx]]
            predicted_labels.append(predicted_label)
            current_word_idx = word_idx

    return list(zip(words, predicted_labels))

# ---------------------
# Example Usage
# ---------------------

example_sentence = "Chicken is great."
result = ner_inference(example_sentence)
print("NER predictions:")
for word, label in result:
    print(f"{word}: {label}")

NER predictions:
Chicken: O
is: O
great.: O


In [ ]:
!pip install  datasets evaluate transformers

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")

# Get the list of labels and number of labels
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

class PreprocessingPipeline:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

    def tokenize_and_align_labels(self, examples):
        tokenized_inputs = self.tokenizer(
            examples['tokens'],
            truncation=True,
            is_split_into_words=True,
            padding="max_length",
            max_length=25,
        )

        all_labels = []

        for i, labels in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            prev_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != prev_word_idx:
                    label_ids.append(labels[word_idx])
                else:
                    label_ids.append(-100)
                prev_word_idx = word_idx
            all_labels.append(label_ids)
        tokenized_inputs['labels'] = all_labels
        return tokenized_inputs

preprocessor = PreprocessingPipeline()
tokenized_datasets = dataset.map(preprocessor.tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
tokenized_datasets['train']['input_ids']

[[101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1943,
  14428,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  26660,
  13329,
  12649,
  15928,
  1820,
  118,
  4775,
  118,
  1659,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1109,
  1735,
  2827,
  1163,
  1113,
  9170,
  1122,
  19786,
  1114,
  1528,
  5566,
  1106,
  11060,
  1106,
  188,
  17315,
  1418,
  2495,
  12913,
  1235,
  6479,
  4959,
  2480,
  102],
 [101,
  1860,
  112,
  188,
  4702,
  1106,
  1103,
  1735,
  1913,
  112,
  188,
  27431,
  3914,
  14651,
  163,
  7635,
  4119,
  1163,
  1113,
  9031,
  11060,
  1431,
  4417,
  8892,
  102],
 [101,
  107,
  1284,
  1202,
  183,
  112,
  189,
  1619,
  1251,
  1216,
  13710,
  1272,
  1195,
  1202,
  183,
  112,
  189,
  1267,
  1251